# Möbius‑Digit‑Thread **Grid Explorer** (gap_scale 12‑20, spacing 3‑5)

**Goal:** sweep a ×9 grid  
(gap_scale ∈ {12,14,16,18,20} × spacing ∈ {3,4,5})  
to pinpoint the density band where **rel_err < 0.01** re‑appears.

Other knobs frozen to the v6 recipe:

* bright tail 400, prime‑gap tail
* tiny exponent k = 4  (tiny = α⁻¹/10⁴)
* φ‑scale micro‑grid 0.003 944 ± 2 × 10⁻⁷ (5 pts)
* seed bits 64 & 128


In [1]:
import mpmath as mp, pandas as pd, itertools, re
mp.mp.dps=260
ALPHA_INV=mp.mpf('137.035999084'); ALPHA=1/ALPHA_INV
PHI=(1+mp.sqrt(5))/2

In [2]:
def digits_of(num,n=700):
    s=str(num).replace('.','')[1:]
    return (s+'0'*n)[:n]
CONST_BANK={'pi':digits_of(mp.pi)}

In [3]:
def primes_upto(n):
    s=[True]*(n+1); s[0:2]=[False,False]
    for i in range(2,int(n**0.5)+1):
        if s[i]: s[i*i:n+1:i]=[False]*len(range(i*i,n+1,i))
    return [i for i,v in enumerate(s) if v]
def prime_gaps(cnt): ps=primes_upto(3000); return [ps[i+1]-ps[i] for i in range(cnt)]

In [4]:
def build_tail(bl=400,sp=5,gs=20):
    gaps=[round(g/gs,4) for g in prime_gaps(bl//sp)]
    tail=[]; gi=iter(gaps)
    for i in range(bl):
        tail.append(1)
        if (i+1)%sp==0: tail.append(next(gi,1))
    return tail
def cf_to_real(co):
    v=mp.mpf(0)
    for a in reversed(co): v=1/(a+v)
    return v

In [5]:
def breath(x,tiny,phi):
    x=phi*PHI/x; x-=tiny; x=1/x/9; x=1/x; return x/9

In [6]:
def const_thread(xdig):
    for L in range(8,3,-1):
        for i in range(len(xdig)-L+1):
            sub=xdig[i:i+L]
            if sub in CONST_BANK['pi']: return f'pi:{sub}'
    return '-'

In [7]:
gap_scales=[12,14,16,18,20]
spacings=[3,4,5]
phi_center=0.003944
phi_vals=[phi_center+i*2e-7 for i in range(-2,3)]
tiny=ALPHA_INV/1e4

rows=[]
for gs,sp in itertools.product(gap_scales,spacings):
    tail=build_tail(sp=sp,gs=gs)
    seed=cf_to_real(tail)
    for bits in (64,128):
        for phi in phi_vals:
            x=breath(seed+tiny,tiny,phi)
            rel=float(abs((x-ALPHA)/ALPHA))
            rows.append(dict(gs=gs,sp=sp,bits=bits,phi=phi,rel_err=rel,
                             thread=const_thread(str(x)[2:402])))
df=pd.DataFrame(rows).sort_values('rel_err').head(60)
df

,gs,sp,bits,phi,rel_err,thread
124,20,3,64,0.003944,1.407926,pi:59230
129,20,3,128,0.003944,1.407926,pi:59230
128,20,3,128,0.003944,1.408001,pi:3993
123,20,3,64,0.003944,1.408001,pi:3993
127,20,3,128,0.003944,1.408075,pi:81097
122,20,3,64,0.003944,1.408075,pi:81097
126,20,3,128,0.003944,1.408150,pi:1328
121,20,3,64,0.003944,1.408150,pi:1328
120,20,3,64,0.003944,1.408224,pi:53594
125,20,3,128,0.003944,1.408224,pi:53594
